# 以泊松分布的方式获取整数列

## 泊松分布公式
$$
P(X=k)=\frac{\lambda^k}{k!}e^{-\lambda}
$$
1. 泊松分布为离散分布
2. 其中$\lambda$是泊松分布的均值
3. 该公式指：在足够多的实验下所得均值为$\lambda$的条件下，实验所得值为`k`的概率

## 泊松脉冲的频率分布
1. 将上述公式中的$\lambda$转换成脉冲的频率均值*rate*
   > 例如设`一组实验`为：将`1s`以`1ms`为单位划分为1000份，每`1ms`实验一次，这组实验共1000次。`进行多组实验`，求多组实验的脉冲次数均值为*rate*。
2. 此时`随机变量X`是一组实验的脉冲次数，随机变量`X`为整数

## 获取符合泊松分布的0/1脉冲序列
1. 从另一个角度看，脉冲的频率分布与脉冲时间间隔分布是可以相互转换的


In [14]:
from torch.distributions import Poisson
import torch

T = 100
rate = 10
# 计算每隔多少interval发放一次
interval_mean = int(T / 10)
interval_generate = Poisson(interval_mean)

assert rate > 0
interval_sum = 0

spike = [0 for _ in range(T)]

while True:
    # 获取下一个脉冲的时间间隔
    interval = interval_generate.sample().item()
    interval_sum += int(interval)
    # print(interval)
    if interval_sum > T+1:
        break
    spike[interval_sum] = 1

print(spike)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]


## bindsnet源码

In [ ]:
def poisson(datum: torch.Tensor, time: int, dt: float = 1.0, **kwargs) -> torch.Tensor:
    # language=rst
    """
    Generates Poisson-distributed spike trains based on input intensity. Inputs must be
    non-negative, and give the firing rate in Hz. Inter-spike intervals (ISIs) for
    non-negative data incremented by one to avoid zero intervals while maintaining ISI
    distributions.
    :param datum: Tensor of shape ``[n_1, ..., n_k]``.
    :param time: Length of Poisson spike train per input variable.
    :param dt: Simulation time step.
    :return: Tensor of shape ``[time, n_1, ..., n_k]`` of Poisson-distributed spikes.
    """
    assert (datum >= 0).all(), "Inputs must be non-negative"

    # Get shape and size of data.
    shape, size = datum.shape, datum.numel()
    datum = datum.flatten()
    time = int(time / dt)

    # Compute firing rates in seconds as function of data intensity,
    # accounting for simulation time step.
    rate = torch.zeros(size)
    rate[datum != 0] = 1 / datum[datum != 0] * (1000 / dt)

    # Create Poisson distribution and sample inter-spike intervals
    # (incrementing by 1 to avoid zero intervals).
    dist = torch.distributions.Poisson(rate=rate)
    intervals = dist.sample(sample_shape=torch.Size([time + 1]))
    intervals[:, datum != 0] += (intervals[:, datum != 0] == 0).float()

    #-------------------------------------------------------------------
    # 与上一部分代码的循环作用等效
    # Calculate spike times by cumulatively summing over time dimension.
    times = torch.cumsum(intervals, dim=0).long()
    times[times >= time + 1] = 0

    # Create tensor of spikes.
    spikes = torch.zeros(time + 1, size).byte()
    spikes[times, torch.arange(size)] = 1
    spikes = spikes[1:]
    #-------------------------------------------------------------------

    return spikes.view(time, *shape)